In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,42556
2,Huelva,Confirmados PDIA 14 días,2487
3,Huelva,Tasa PDIA 14 días,"484,63472143734043"
4,Huelva,Confirmados PDIA 7 días,877
5,Huelva,Tasa PDIA 7 dias,"170,89853264999903"
6,Huelva,Total Confirmados,42755
7,Huelva,Curados,36083
8,Huelva,Fallecidos,397


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  42556.0


/tmp/ipykernel_5384/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11878.0


/tmp/ipykernel_5384/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_5384/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_5384/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_5384/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 634 personas en los últimos 7 días 

Un positivo PCR cada 204 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,42556.0,877.0,2487.0,513170.0,170.898533,484.634721,143.0
Huelva-Costa,25039.0,535.0,1496.0,289548.0,184.770746,516.667357,105.0
Huelva (capital),11878.0,227.0,704.0,143837.0,157.817530,489.442911,42.0
Condado-Campiña,13356.0,272.0,801.0,156231.0,174.101171,512.702345,25.0
Isla Cristina,2883.0,71.0,159.0,21393.0,331.884261,743.233768,18.0
Sierra de Huelva-Andévalo Central,3770.0,65.0,180.0,67391.0,96.452048,267.097980,16.0
Ayamonte,1852.0,58.0,155.0,21104.0,274.829416,734.457923,14.0
Cartaya,1715.0,59.0,112.0,20083.0,293.780810,557.685605,12.0
Punta Umbría,1195.0,21.0,56.0,15355.0,136.763269,364.702051,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Sanlúcar de Guadiana,31.0,1.0,17.0,403.0,248.138958,4218.362283,0.0
Cañaveral de León,18.0,3.0,6.0,396.0,757.575758,1515.151515,0.0
Escacena del Campo,207.0,5.0,24.0,2287.0,218.627022,1049.409707,0.0
Villarrasa,214.0,4.0,23.0,2211.0,180.913614,1040.253279,0.0
Alájar,25.0,6.0,7.0,759.0,790.513834,922.266140,1.0
Paterna del Campo,188.0,11.0,29.0,3457.0,318.194967,838.877640,0.0
Trigueros,517.0,29.0,65.0,7862.0,368.862885,826.761638,1.0
Bonares,436.0,17.0,49.0,6060.0,280.528053,808.580858,0.0
Lucena del Puerto,343.0,6.0,26.0,3261.0,183.992640,797.301441,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Sanlúcar de Guadiana,31.0,1.0,17.0,403.0,248.138958,4218.362283,0.0,0.058824
Minas de Riotinto,166.0,1.0,7.0,3812.0,26.232949,183.630640,0.0,0.142857
Campillo (El),98.0,2.0,13.0,2024.0,98.814229,642.292490,0.0,0.153846
Villalba del Alcor,492.0,1.0,6.0,3366.0,29.708853,178.253119,0.0,0.166667
Villarrasa,214.0,4.0,23.0,2211.0,180.913614,1040.253279,0.0,0.173913
Galaroza,54.0,1.0,5.0,1382.0,72.358900,361.794501,1.0,0.200000
Calañas,240.0,1.0,5.0,2768.0,36.127168,180.635838,0.0,0.200000
Escacena del Campo,207.0,5.0,24.0,2287.0,218.627022,1049.409707,0.0,0.208333
Lucena del Puerto,343.0,6.0,26.0,3261.0,183.992640,797.301441,0.0,0.230769
